In [ ]:
import pandas as pd
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### This cell sets the path to the data depending on the user.

- Just replace the ellipses with where your `ER131-Project` folder is in your Google Drive and change the user

In [ ]:
GOOGLE_DRIVE_BASE_PATH = "/content/drive/MyDrive/"
user = "Samadi" # change before running the notebook

if user == "Julia":
    DATA_PATH = GOOGLE_DRIVE_BASE_PATH + "..." + "data/"
elif user == "Matthew":
    DATA_PATH = GOOGLE_DRIVE_BASE_PATH + "Sustainability/ER131-Project/" + "data/"
elif user == "Samadi":
    DATA_PATH = GOOGLE_DRIVE_BASE_PATH + "ER131-Project/" + "data/"
elif user == "Shaye":
    DATA_PATH = GOOGLE_DRIVE_BASE_PATH + "ER131-Project/" + "data/"
    
print("User: {}\nPath to data: {}".format(user, DATA_PATH))

User: Samadi
Path to data: /content/drive/MyDrive/ER131-Project/data/


## Cleaning CalRecycle Waste Data

### Combine all individual county files into a single df

In [ ]:
def filter_waste_origin_df(df, county):
    """Drop quarterly values in the csv and only keep yearly values
    Input:
        - calrecycle df
        - county (string)
    
    Ouput:
        - df with year, county, disposal ton
    """
    columns_to_keep = ["Report Year", "Diposal Ton", "County"]
    df["County"] = county
    missing_data = df["Report Year"].isnull()
    df = df[~missing_data]
    return df[columns_to_keep]

In [ ]:
calrecycle_data_path = DATA_PATH + "calrecycle-waste-production-data/"
county_waste_production = []
for f in os.listdir(calrecycle_data_path):
    filename, ext = os.path.splitext(f)
    county = filename.split("_waste_production")[0]
    county = county.replace("_", " ")
    print("[INFO] filtering county file: ", os.path.join(calrecycle_data_path, f))
    try:
        df = pd.read_excel(os.path.join(calrecycle_data_path, f), skiprows=3, skipfooter=2)
        filtered_df = filter_waste_origin_df(df, county)
        county_waste_production.append(filtered_df)
    except Exception as e:
        print("[DEBUG] Skipping: ", county)

county_waste_production_df = pd.concat(county_waste_production)

[INFO] filtering county file:  /content/drive/MyDrive/ER131-Project/data/calrecycle-waste-production-data/Alameda_waste_production.xlsx
[INFO] filtering county file:  /content/drive/MyDrive/ER131-Project/data/calrecycle-waste-production-data/Alpine_waste_production.xlsx
[INFO] filtering county file:  /content/drive/MyDrive/ER131-Project/data/calrecycle-waste-production-data/Amador_waste_production.xlsx
[INFO] filtering county file:  /content/drive/MyDrive/ER131-Project/data/calrecycle-waste-production-data/Butte_waste_production.xlsx
[INFO] filtering county file:  /content/drive/MyDrive/ER131-Project/data/calrecycle-waste-production-data/Calaveras_waste_production.xlsx
[INFO] filtering county file:  /content/drive/MyDrive/ER131-Project/data/calrecycle-waste-production-data/Colusa_waste_production.xlsx
[INFO] filtering county file:  /content/drive/MyDrive/ER131-Project/data/calrecycle-waste-production-data/Contra_Costa_waste_production.xlsx
[INFO] filtering county file:  /content/drive/

In [ ]:
county_waste_production_df.rename(columns={"Report Year": "Year", 
                                           "Diposal Ton": "Waste Produced (Tons)"}, inplace=True)
county_waste_production_df.head()

Year  Waste Produced (Tons)   County
0    1995.0             1583135.05  Alameda
65   1996.0             1510874.25  Alameda
130  1997.0             1593570.29  Alameda
195  1998.0             1671466.25  Alameda
260  1999.0             1733182.35  Alameda

### Dealing with null values

- drop these from the data because it doesn't make sense to fit a model without a target variable

In [ ]:
num_null_values = county_waste_production_df.shape[0] - county_waste_production_df.dropna().shape[0]
print("number of null values: ", num_null_values)

number of null values:  13


In [ ]:
no_waste_data = county_waste_production_df[county_waste_production_df.isnull().any(axis=1)]
no_waste_data

Year  Waste Produced (Tons)     County
117  2008.0                    NaN  Del Norte
13   1996.0                    NaN      Modoc
39   1998.0                    NaN      Modoc
52   1999.0                    NaN      Modoc
78   2001.0                    NaN      Modoc
91   2002.0                    NaN      Modoc
104  2003.0                    NaN      Modoc
130  2005.0                    NaN      Modoc
143  2006.0                    NaN      Modoc
156  2007.0                    NaN      Modoc
169  2008.0                    NaN      Modoc
195  2010.0                    NaN      Modoc
207  2018.0                    NaN     Sierra

In [ ]:
county_waste_production_df.dropna(inplace=True)

### Tests - sanity check that nothing went wrong during data cleaning

- Total number of counties (excluding Sutter) * total number of years = 58 - 1 * (2019 - 1995 + 1) - dropped null values = total number of observations in the final df
- Alameda County 2019 total waste produced = 1,465,264	
- LA County 2012 total waste produced = 8,141,712
- Orange County 2005 total waste produced = 4,290,785

In [ ]:
def get_feature(df, county, year, feature):
    """Testing function that gets a specific feature value from the dataframe from a specific year and county
    """
    county_condition = df["County"] == county
    year_condition = df["Year"] == year
    return df.loc[county_condition & year_condition, feature]

assert ~county_waste_production_df.isnull().values.any()
# assert all(~county_waste_production_df.isnull()) this comparison does not work
assert (58 - 1) * (2019 - 1995 + 1) - num_null_values == county_waste_production_df.shape[0]
assert np.isclose(1465264, get_feature(county_waste_production_df, "Alameda", 2019, "Waste Produced (Tons)"))
assert np.isclose(8141712, get_feature(county_waste_production_df, "Los Angeles", 2012, "Waste Produced (Tons)"))
assert np.isclose(4290785, get_feature(county_waste_production_df, "Orange", 2005, "Waste Produced (Tons)"))

In [ ]:
# def get_feature(df, feature, **kwargs):
#     """Gets a specific feature from the dataframe
#     """
#     year = kwargs["year"]
#     county = kwargs["county"]
#     feature = kwargs["feature"]
#     county_condition = df["County"] == county
#     year_condition = df["Year"] == year
#     return df.loc[county_condition & year_condition, feature]

# get_feature(county_waste_production_df, **{"feature": "Waste Produced (Tons)", "year": 2012, "county":"Los Angeles"})

### Save cleaned waste production csv after passing tests

In [ ]:
county_waste_production_df.to_csv(DATA_PATH + "calrecycle_waste_production_per_county.csv", index=False)

---

## Cleaning Census Population Data

### 2010-2019

In [ ]:
county_population_2010_2019 = pd.read_excel(DATA_PATH + "ca-finance-dpt-county-populations-2010-2019.xlsx", 
                                            sheet_name="Table 1 County State", header=1, skipfooter=1)
county_population_2010_2019.rename(columns={"COUNTY":"County"}, inplace=True)
county_population_2010_2019["County"] = county_population_2010_2019.County.str.strip()
county_population_2010_2019.head()

County  4/1/2010  1/1/2011  ...  1/1/2018  1/1/2019  1/1/2020
0    Alameda   1510271   1526110  ...   1655306   1664783   1670834
1     Alpine      1175      1169  ...      1159      1149      1142
2     Amador     38091     37173  ...     37563     37820     37676
3      Butte    220000    220620  ...    226374    221521    210291
4  Calaveras     45578     45540  ...     45155     45085     45023

[5 rows x 12 columns]

In [ ]:
county_population_2010_2019.set_index("County", inplace=True)
col_names_date = county_population_2010_2019.columns 
col_names_year = [int(date[-4:]) for date in col_names_date]
county_population_2010_2019.columns = col_names_year
county_population_2010_2019.reset_index(inplace=True)
county_population_2010_2019.head()

County     2010     2011     2012  ...     2017     2018     2019     2020
0    Alameda  1510271  1526110  1545917  ...  1646711  1655306  1664783  1670834
1     Alpine     1175     1169     1166  ...     1161     1159     1149     1142
2     Amador    38091    37173    36777  ...    36928    37563    37820    37676
3      Butte   220000   220620   221340  ...   225643   226374   221521   210291
4  Calaveras    45578    45540    45496  ...    45170    45155    45085    45023

[5 rows x 12 columns]

In [ ]:
county_population_2010_2019 = county_population_2010_2019.melt(id_vars="County", var_name='Year', value_name="Population")
county_population_2010_2019.head()

County  Year  Population
0    Alameda  2010     1510271
1     Alpine  2010        1175
2     Amador  2010       38091
3      Butte  2010      220000
4  Calaveras  2010       45578

### 2000-2009

In [ ]:
county_population_2000_2009 = pd.read_excel(DATA_PATH + "ca-finance-dpt-county-populations-2000-2009.xls", 
                                            sheet_name="Table 1 County State", 
                                            header=3, skipfooter=2)
county_population_2000_2009.rename(columns={"COUNTY":"County"}, inplace=True)
county_population_2000_2009["County"] = county_population_2000_2009.County.str.strip()
county_population_2000_2009.dropna(how='all', inplace=True)
county_population_2000_2009.head()

County  2000-04-01 00:00:00  ...  2010-01-01 00:00:00  2010-04-01 00:00:00
1    Alameda            1443939.0  ...            1509240.0            1510271.0
2     Alpine               1208.0  ...               1178.0               1175.0
3     Amador              35100.0  ...              38011.0              38091.0
4      Butte             203171.0  ...             219967.0             220000.0
5  Calaveras              40554.0  ...              45602.0              45578.0

[5 rows x 13 columns]

In [ ]:
county_population_2000_2009.set_index("County", inplace=True)
col_names_date = county_population_2000_2009.columns 
col_names_year = [date.year for date in col_names_date]
county_population_2000_2009.columns = col_names_year
county_population_2000_2009.reset_index(inplace=True)
county_population_2000_2009.drop(columns=[2010], inplace=True) # drop 2010 bc already included in 2010-2019
county_population_2000_2009.head()

County       2000       2001  ...       2007       2008       2009
0    Alameda  1443939.0  1457185.0  ...  1470622.0  1484085.0  1497799.0
1     Alpine     1208.0     1220.0  ...     1252.0     1228.0     1194.0
2     Amador    35100.0    35495.0  ...    38025.0    37975.0    37884.0
3      Butte   203171.0   204591.0  ...   216401.0   217801.0   218887.0
4  Calaveras    40554.0    41042.0  ...    45477.0    45670.0    45632.0

[5 rows x 11 columns]

In [ ]:
county_population_2000_2009 = county_population_2000_2009.melt(id_vars="County", var_name='Year', value_name="Population")
county_population_2000_2009.head()

County  Year  Population
0    Alameda  2000   1443939.0
1     Alpine  2000      1208.0
2     Amador  2000     35100.0
3      Butte  2000    203171.0
4  Calaveras  2000     40554.0

### append 2000-2009 & 2010-2019

In [ ]:
county_population_df = county_population_2000_2009.append(county_population_2010_2019)
county_population_df.reset_index(inplace=True, drop=True)
county_population_df.head()

County  Year  Population
0    Alameda  2000   1443939.0
1     Alpine  2000      1208.0
2     Amador  2000     35100.0
3      Butte  2000    203171.0
4  Calaveras  2000     40554.0

In [ ]:
county_population_df.Year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=object)

### Tests

- Alameda County 2019 population = 1,664,783 
- LA County 2012 population = 9,956,882 
- Orange County 2005 population = 2,956,847

In [ ]:
def get_feature(df, county, year, feature):
    """Testing function that gets a specific feature value from the dataframe from a specific year and county
    """
    county_condition = df["County"] == county
    year_condition = df["Year"] == year
    return df.loc[county_condition & year_condition, feature]

assert ~county_population_df.isnull().values.any()
assert np.isclose(1664783, get_feature(county_population_df, "Alameda", 2019, "Population"))
assert np.isclose(9956882, get_feature(county_population_df, "Los Angeles", 2012,  "Population"))
assert np.isclose(2956847, get_feature(county_population_df, "Orange", 2005, "Population"))

In [ ]:
### Save cleaned population csv after passing tests
county_population_df.to_csv(DATA_PATH + "ca-finance-dpt-county-populations-2000-2020.csv", index=False)

---

## Cleaning electricity usage data

In [ ]:
electricity_usage_per_county = pd.read_csv(DATA_PATH + "cec-electricity-by-county-full.csv")
electricity_usage_per_county = electricity_usage_per_county[electricity_usage_per_county.Sector == "Residential"]
electricity_usage_per_county["County"] = electricity_usage_per_county.County.str.lower().str.title()
electricity_usage_per_county.drop(columns=["Total Usage", "Sector"], inplace=True)
electricity_usage_per_county.head()

County         2019         2018  ...         1992         1991         1990
1     Alameda  3064.781376  2945.804917  ...  2465.000074  2515.209970  2498.265626
4      Alpine    10.131788     9.420910  ...     5.808735     6.165242     6.036978
7      Amador   140.689119   136.895373  ...   105.907059   108.875556   105.093388
10      Butte   669.328499   717.743260  ...   555.149304   553.073852   547.901570
13  Calaveras   205.365353   201.833277  ...   140.066797   143.201069   138.299372

[5 rows x 31 columns]

In [ ]:
electricity_usage_per_county = electricity_usage_per_county.melt(id_vars="County", var_name='Year', value_name="Electricity Usage (GWh)")
electricity_usage_per_county["Year"] = pd.to_numeric(electricity_usage_per_county["Year"])
electricity_usage_per_county.head()

County  Year  Electricity Usage (GWh)
0    Alameda  2019              3064.781376
1     Alpine  2019                10.131788
2     Amador  2019               140.689119
3      Butte  2019               669.328499
4  Calaveras  2019               205.365353

In [ ]:
def get_feature(df, county, year, feature):
    """Testing function that gets a specific feature value from the dataframe from a specific year and county
    """
    county_condition = df["County"] == county
    year_condition = df["Year"] == year
    return df.loc[county_condition & year_condition, feature]

assert ~electricity_usage_per_county.isnull().values.any()
assert np.isclose(3064.781376, get_feature(electricity_usage_per_county, "Alameda", 2019, "Electricity Usage (GWh)"))
assert np.isclose(21076.22492, get_feature(electricity_usage_per_county, "Los Angeles", 2012,  "Electricity Usage (GWh)"))
assert np.isclose(6717.053806, get_feature(electricity_usage_per_county, "Orange", 2005, "Electricity Usage (GWh)"))

In [ ]:
### Save cleaned electricity usage csv after passing tests
electricity_usage_per_county.to_csv(DATA_PATH + "cec-electricity-by-county-cleaned.csv", index=False)

## Cleaning House Hold Type

The following data frame comes from the CA State Association of Counties. It includes the number of moblie, multi-family, and single family housing units in each county from 1991-2018. Below, I pivoted the table below such that each row represents a county and a year and there is a column for each housing type. This will make the data easier to combine to the main dataframe. 

In [ ]:
housing_type = pd.read_csv(DATA_PATH+'housing_units_by_county.csv')
housing_type_by_county = pd.DataFrame(housing_type.pivot(index=['County','Year'], columns = 'Type').reset_index().to_records())
housing_type_by_county.columns = ['index','County', 'Year', 'Mobile Homes (units)', 'Multi-Family (units)', 'Single Family (units)']
housing_type_by_county.drop('index', axis = 1, inplace = True)
housing_type_by_county.head()

County  Year  ... Multi-Family (units) Single Family (units)
0  Alameda  1991  ...             198,944               301,805 
1  Alameda  1992  ...             199,339               301,569 
2  Alameda  1993  ...             199,902               303,587 
3  Alameda  1994  ...             200,184               306,280 
4  Alameda  1995  ...             199,951               309,033 

[5 rows x 5 columns]

In [ ]:
housing_type_by_county.to_csv(DATA_PATH + 'housing_type_by_county.csv', index = False)

In [ ]:
years = np.arange(2000,2020)
for year in years:
  print(len(housing_type_by_county.loc[housing_type_by_county['Year'] == year].County.unique()))

58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
0


There does not seem to be any redundant entries or typos

Now I will check to see if there are any null values. There are none!

In [ ]:
housing_type_by_county.isnull().values.any()

False

In [ ]:
len(housing_type_by_county['County'].unique())

58

## Cleaning Expenditure Data



In [ ]:
 income = pd.read_csv(DATA_PATH+'income_by_county.csv')
 income = income[['County', 'Taxable Year', 'Median Income']]
 income = income.loc[income['Taxable Year'] > 1999]
 np.sort(income['Taxable Year'].unique())
 income.dtypes
 len(income['County'].unique())

64

In [ ]:
income_2018 = pd.read_csv(DATA_PATH+'original_income_data_2000_2018.csv')
income_2018['Taxable Year'] = income_2018['Taxable Year'].replace(['2,018'], '2018')
income_2018['Median Income'] = income_2018['Median Income'].replace([','], '')
income_2018['Taxable Year'] = income_2018['Taxable Year'].astype(int)
#income_2018['Median Income'] = income_2018['Median Income'].astype(int)

#income_2018 = income_2018.loc[income_2018['Taxable Year'] > 2017]
income_2018
#np.sort(income['Taxable Year'].unique())

County  Taxable Year Median Income
0           Modoc          2009         27499
1     Nonresident          2005         58612
2           Modoc          2003         25749
3      Sacramento          2007         36623
4       San Diego          2003         32410
...           ...           ...           ...
1151       Tulare          2018        47,518
1152     Tuolumne          2018        56,493
1153      Ventura          2018        84,017
1154         Yolo          2018        65,923
1155         Yuba          2018        52,624

[1156 rows x 3 columns]

In [ ]:
income_2018 = pd.read_excel(DATA_PATH+'2018 California County Inc..xlsx')
income_2018['County'] = income_2018['County'].str.replace('County', '')
income_2018 = income_2018.dropna()
income_2018 = income_2018[['County', 'Year', 'Median Income']]
income_2018.head()

County  Year  Median Income
0    Alameda  2018          92574
1     Alpine  2018          64688
2     Amador  2018          61198
3      Butte  2018          48443
4  Calaveras  2018          58151

In [ ]:
#  income.to_csv(DATA_PATH+'income_data.csv', index = False)

In [ ]:
Median_Income_by_Yr = income.groupby('Taxable Year').agg({ "Median Income": np.mean})
Median_Income_by_County = income.groupby('County').agg({ "Median Income": np.mean})
income_sorted = income.sort_values(by = ['Taxable Year', 'County'])
income_sorted.rename(columns={"Taxable Year": "Year"}, errors='raise', inplace = True)
income_sorted["County"] = income_sorted.County.str.strip()
len(income_sorted.County.unique())

64

In [ ]:
inc_2019 = pd.read_excel(DATA_PATH + '2019 California County Inc. (2).xlsx')
inc_2019.rename(columns={"Area Median Income:": "Median Income"}, errors='raise', inplace = True)
inc_2019['County'] = inc_2019['County'].str.replace('County', '')
inc_2019 = inc_2019.dropna()
inc_2019 = inc_2019[['County', 'Year', 'Median Income']]
inc_2019

County    Year  Median Income
0           Alameda   2019.0       111700.0
1            Alpine   2019.0        94900.0
2            Amador   2019.0        73600.0
3             Butte   2019.0        66500.0
4         Calaveras   2019.0        75300.0
5            Colusa   2019.0        64800.0
6      Contra Costa   2019.0       111700.0
7         Del Norte   2019.0        64800.0
8         El Dorado   2019.0        83600.0
9            Fresno   2019.0        64800.0
10            Glenn   2019.0        64800.0
11         Humboldt   2019.0        64800.0
12         Imperial   2019.0        64800.0
13             Inyo   2019.0        72700.0
14             Kern   2019.0        64800.0
15            Kings   2019.0        64800.0
16             Lake   2019.0        64800.0
17           Lassen   2019.0        69400.0
18      Los Angeles   2019.0        73100.0
19          Madera    2019.0        64800.0
20           Marin    2019.0       136800.0
21         Mariposa   2019.0        65500.0
22        Mendocino   2019.0        64800.0
23           Merced   2019.0        64800.0
24            Modoc   2019.0        64800.0
25             Mono   2019.0        81200.0
26         Monterey   2019.0        74100.0
27             Napa   2019.0       100400.0
28           Nevada   2019.0        85100.0
29           Orange   2019.0        97900.0
30           Placer   2019.0        83600.0
31           Plumas   2019.0        70700.0
32        Riverside   2019.0        69700.0
33       Sacramento   2019.0        83600.0
34       San Benito   2019.0        84500.0
35   San Bernardino   2019.0        69700.0
36        San Diego   2019.0        86300.0
37    San Francisco   2019.0       136800.0
38      San Joaquin   2019.0        71400.0
39  San Luis Obispo   2019.0        87500.0
40        San Mateo   2019.0       136800.0
41    Santa Barbara   2019.0        79600.0
42      Santa Clara   2019.0       131400.0
43       Santa Cruz   2019.0        98000.0
44           Shasta   2019.0        64800.0
45           Sierra   2019.0        71800.0
46         Siskiyou   2019.0        64800.0
47           Solano   2019.0        85700.0
48           Sonoma   2019.0        93300.0
49       Stanislaus   2019.0        64800.0
50           Sutter   2019.0        64800.0
51           Tehama   2019.0        64800.0
52          Trinity   2019.0        64800.0
53           Tulare   2019.0        64800.0
54         Tuolumne   2019.0        66700.0
55          Ventura   2019.0        97800.0
56             Yolo   2019.0        87900.0
57             Yuba   2019.0        64800.0

In [ ]:
median_income = income_sorted.append(income_2018)
median_income.reset_index(inplace=True, drop=True)
median_income = median_income.append(inc_2019)
median_income.reset_index(inplace=True, drop=True)
median_income['County'] = median_income['County'].str.strip()
median_income

County    Year  Median Income
0       Alameda  2000.0        37580.0
1        Alpine  2000.0        29333.0
2        Amador  2000.0        31806.0
3         Butte  2000.0        24513.0
4     Calaveras  2000.0        31420.0
...         ...     ...            ...
1209     Tulare  2019.0        64800.0
1210   Tuolumne  2019.0        66700.0
1211    Ventura  2019.0        97800.0
1212       Yolo  2019.0        87900.0
1213       Yuba  2019.0        64800.0

[1214 rows x 3 columns]

In [ ]:
median_income.Year.unique()

array([2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008.,
       2009., 2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017.,
       2018., 2019.])

In [ ]:
median_income.County.unique()

array(['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Nonresident', 'Orange', 'Placer', 'Plumas',
       'Resident Out of State', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne',
       'Unallocated', 'Ventura', 'Yolo', 'Yuba', 'Nonresident20',
       'Resident Out-of-State19', 'Resident Out-of-State'], dtype=object)

In [ ]:
median_income.to_csv(DATA_PATH+'median_income_data.csv', index = False)

# Population Density Variable





In [ ]:
county_population = pd.read_csv(DATA_PATH + "ca-finance-dpt-county-populations-2000-2020.csv")
county_population.head(3)

County  Year  Population
0  Alameda  2000   1443939.0
1   Alpine  2000      1208.0
2   Amador  2000     35100.0

In [ ]:
area = pd.read_excel(DATA_PATH+'Land area in square miles, 2010 - (Square Miles).xlsx')

area.rename(columns = {'Value': 'Square Miles Area'}, inplace = True)
pop_density_df = county_population.merge(area, on = ['County'], how = 'inner')
pop_density = pop_density_df['Population']/pop_density_df['Square Miles Area'] 
pop_density_df['Population Density (Person per Square Mile)'] = pop_density
pop_density_df.sort_values(by = ['Year'])



County  ...  Population Density (Person per Square Mile)
0           Alameda  ...                                  1953.856459
756       San Diego  ...                                   668.904325
189          Fresno  ...                                   134.173941
777   San Francisco  ...                                 16572.071688
798     San Joaquin  ...                                   405.081505
...             ...  ...                                          ...
671          Plumas  ...                                     7.152258
650          Placer  ...                                   286.928309
629          Orange  ...                                  4040.542899
860       San Mateo  ...                                  1724.412926
1217           Yuba  ...                                   124.852811

[1218 rows x 5 columns]

In [ ]:
len(pop_density_df['County'].unique())

58

In [ ]:
pop_density_df.to_csv(DATA_PATH+'pop_density_df.csv', index = False)

---

# Merging all individual feature dfs 

> add additional features in this section

## Load in all the CSVs

In [ ]:
county_waste_production = pd.read_csv(DATA_PATH + "calrecycle_waste_production_per_county.csv")
county_waste_production.head(3)
len(county_waste_production.County.unique())

57

In [ ]:
county_population = pd.read_csv(DATA_PATH + "ca-finance-dpt-county-populations-2000-2020.csv")
county_population.head(3)

County  Year  Population
0  Alameda  2000   1443939.0
1   Alpine  2000      1208.0
2   Amador  2000     35100.0

In [ ]:
electricity_usage_county = pd.read_csv(DATA_PATH + "cec-electricity-by-county-cleaned.csv")
electricity_usage_county.head(3)

County  Year  Electricity Usage (GWh)
0  Alameda  2019              3064.781376
1   Alpine  2019                10.131788
2   Amador  2019               140.689119

In [ ]:
housing_type_by_county = pd.read_csv(DATA_PATH + 'housing_type_by_county_1991_2019.csv')
housing_type_by_county.head()

County  Year  ...  Multi-Family (units)  Single Family (units)
0  Alameda  1991  ...                198944                 301805
1  Alameda  1992  ...                199339                 301569
2  Alameda  1993  ...                199902                 303587
3  Alameda  1994  ...                200184                 306280
4  Alameda  1995  ...                199951                 309033

[5 rows x 5 columns]

In [ ]:
population_density = pd.read_csv(DATA_PATH + 'pop_density_df.csv')
population_density.head()

County  ...  Population Density (Person per Square Mile)
0  Alameda  ...                                  1953.856459
1  Alameda  ...                                  1971.780195
2  Alameda  ...                                  1985.146545
3  Alameda  ...                                  1986.268301
4  Alameda  ...                                  1984.258883

[5 rows x 5 columns]

In [ ]:
median_income = pd.read_csv(DATA_PATH + 'median_income_data.csv')
median_income.head()

County    Year  Median Income
0    Alameda  2000.0        37580.0
1     Alpine  2000.0        29333.0
2     Amador  2000.0        31806.0
3      Butte  2000.0        24513.0
4  Calaveras  2000.0        31420.0

## Merge dataframes

In [ ]:
#changed from county_population to population_density
complete_feature_df = county_waste_production.merge(population_density, on=["County", "Year"], how="inner")
complete_feature_df.head()

Year  ...  Population Density (Person per Square Mile)
0  2000.0  ...                                  1953.856459
1  2001.0  ...                                  1971.780195
2  2002.0  ...                                  1985.146545
3  2003.0  ...                                  1986.268301
4  2004.0  ...                                  1984.258883

[5 rows x 6 columns]

In [ ]:
complete_feature_df = complete_feature_df.merge(electricity_usage_county, on=["County", "Year"], how="inner")
complete_feature_df.head(3)

Year  ...  Electricity Usage (GWh)
0  2000.0  ...              2926.106226
1  2001.0  ...              2745.201964
2  2002.0  ...              2811.610305

[3 rows x 7 columns]

In [ ]:
complete_feature_df = complete_feature_df.merge(housing_type_by_county, on = ['County','Year'], how = 'inner')
complete_feature_df.head(3)

Year  Waste Produced (Tons)  ... Multi-Family (units)  Single Family (units)
0  2000.0             1676429.25  ...               203132                 328399
1  2001.0             1629208.38  ...               204237                 332390
2  2002.0             1596803.00  ...               205444                 335234

[3 rows x 10 columns]

In [ ]:
complete_feature_df.Year.unique()

array([2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008.,
       2009., 2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017.,
       2018., 2019.])

In [ ]:
len(complete_feature_df.County.unique())

57

In [ ]:
# complete_feature_df = complete_feature_df.merge(median_income, on = ['Year', 'County'], how = 'inner')
# complete_feature_df.head(3)

In [ ]:
complete_feature_df.Year.unique()

array([2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008.,
       2009., 2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017.,
       2018., 2019.])

In [ ]:
def get_feature(df, county, year, feature):
    """Testing function that gets a specific feature value from the dataframe from a specific year and county
    """
    county_condition = df["County"] == county
    year_condition = df["Year"] == year
    return df.loc[county_condition & year_condition, feature]

## Tests that should pass before doing any EDA / modeling analysis

In [ ]:
# check we have all CA counties
num_counties = 58

assert num_counties-1 == len(complete_feature_df.County.unique()) # minus Sutter bc it has no waste data

# check there are no null values in the data
assert ~complete_feature_df.isnull().values.any()

# check time range of our data is consistent (2000 - 2019) - depends on what intersection of years we have btween all our data
assert all(complete_feature_df.Year < 2020)
assert all(complete_feature_df.Year > 1999)

# check waste tests pass
assert np.isclose(1465264, get_feature(complete_feature_df, "Alameda", 2019, "Waste Produced (Tons)"))
assert np.isclose(8141712, get_feature(complete_feature_df, "Los Angeles", 2012, "Waste Produced (Tons)"))
assert np.isclose(4290785, get_feature(complete_feature_df, "Orange", 2005, "Waste Produced (Tons)"))

# check population tests pass
assert np.isclose(1664783, get_feature(complete_feature_df, "Alameda", 2019, "Population"))
assert np.isclose(9956882, get_feature(complete_feature_df, "Los Angeles", 2012,  "Population"))
assert np.isclose(2956847, get_feature(complete_feature_df, "Orange", 2005, "Population"))

# check electricity tests pass
assert np.isclose(3064.781376, get_feature(complete_feature_df, "Alameda", 2019, "Electricity Usage (GWh)"))
assert np.isclose(21076.22492, get_feature(complete_feature_df, "Los Angeles", 2012,  "Electricity Usage (GWh)"))
assert np.isclose(6717.053806, get_feature(complete_feature_df, "Orange", 2005, "Electricity Usage (GWh)"))

# check 

## Save complete feature df after passing tests


In [ ]:
# sorts by year, then by county so it's in yearly ascending order and then alphabetical by county
complete_feature_df.sort_values(["Year", "County"], inplace=True)

In [ ]:
complete_feature_df.drop(columns=["Square Miles Area"], inplace=True)
complete_feature_df['Year'].unique()

array([2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008.,
       2009., 2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017.,
       2018., 2019.])

In [ ]:
complete_feature_df.to_csv(DATA_PATH + "complete_feature_df.csv", index=False)

In [ ]:
features_and_inc = median_income.merge(complete_feature_df, on = ['County', 'Year'], how = 'inner')
#len(new['Year'].unique())
len(features_and_inc['County'].unique())
#features_and_inc
#features_and_inc['Year'].unique()

57

In [ ]:
median_income['Year'].unique()

array([2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008.,
       2009., 2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017.,
       2018., 2019.])

In [ ]:
features_and_inc.to_csv(DATA_PATH + "complete_feature_and_income.csv", index=False)